<a href="https://colab.research.google.com/github/kassemm/LAB3AV/blob/dependabot%2Fpip%2Fdocs%2Fpygments-2.7.4/Lab3/Lab3_Highway_DQN_rlagents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab3 is based on highway-env https://github.com/eleurent/highway-env. In 
envs/highway_env.py, the vehicle is rewarded for reaching a high speed, staying on the rightmost lanes and avoiding collisions for in highway driving (highway-env). Your task is to add an additional term to penalize changing lanes. (Ideally we should penalize changing lanes to the left more than changing to the right, but this is a simplified exercise.) Please refer to envs/roundabout_env for reference.

Steps:

1) Fork the highway-env repository to your own GitHub account, and modify highway_env.py. (if you are not familiar with GitHub, I recommend installing GitHub Desktop https://desktop.github.com.)

2) Replace 
!pip install highway-env 
with path to your own repository:
!pip install git+https://github.com/USERNAME/highway-env

3) Train your agent for different values of lane_change_reward, and different environments with varying complexity, starting from the simplest:

env.config["lane_change_reward"] = 0

env.config["vehicles_count"] = 0

env.config["lanes_count"] = 2

env.config["initial_lane_id"] = 0

In this simple environment, during testing the vehicle should switch to the right lane quickly and stay in the right lane. (It is possible for it to stay in the left lane during testing if your training num_episodes is too small, since the agent may not have had a chance to try the lane change action during training.)
Try different values of lane_change_reward (e.g., -0.1, -1, -10, etc. note that it must be negative), and different numbers of vehicles/lanes. Make lane_change_reward very negative until you observe that the vehicle prefers *collision* to *lane change to avoid collision*. Submit your IPython Notebook, and a short report (or text blocks within the Notebook) describing the different configurations you have tried, and your observations for each config, including two configs that differ only in the lane_change_reward, one with no or rare collisions, one with frequent collisions. Please share your Colab link with the trained model and the recorded videos for the testing in IPython Notebook, so we do not have to re-train the models (unless you are not using Colab, then upload your IPython Notebook or .py file to Canvas). 

Notes: 

1) More complex environments demand more training time (more num_episodes), so do not make your environment more complex than necessary. You may try increasing the  vehicles_density and reward_speed_range to increase the chances of collision. You may also find "Script to Stop Google Colab From Disconnecting" to be helpful.

2) You may want to tune DQN hyperparams in configs/HighwayEnv/agents/DQNAgent/dqn.json, e.g., discount factor gamma, and epsilon greedy exploration params tau, temperature, final_temperature for different num_episodes.

3) The error message "fatal: destination path 'highway-env' already exists and is not an empty directory" in the first code cell can be safely ignored.


# Training a DQN on `highway-v0` with rlagents
## Import requirements

In [7]:
# Environment
##!pip install highway-env
!pip install git+https://github.com/kassemm/highway-env
import gym
import highway_env

# Agent
##!pip install git+https://github.com/eleurent/rl-agents
!pip install git+https://github.com/kassemm/rl-agents


# Visualisation utils
import sys
%load_ext tensorboard
!pip install tensorboardx gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg
#!git clone https://github.com/eleurent/highway-env.git
!git clone https://github.com/kassemm/highway-env.git
sys.path.insert(0, '/content/highway-env/scripts/')
from utils import show_videos


from rl_agents.trainer.evaluation import Evaluation
from rl_agents.agents.common.factory import load_agent, load_environment

# Get the environment and agent configurations from the rl-agents repository
##!git clone https://github.com/eleurent/rl-agents.git
!git clone https://github.com/kassemm/rl-agents.git



  Cloning https://github.com/kassemm/highway-env to /tmp/pip-req-build-_s09cg0f
  Running command git clone -q https://github.com/kassemm/highway-env /tmp/pip-req-build-_s09cg0f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... canceled
ERROR: Operation cancelled by user
  Cloning https://github.com/kassemm/rl-agents to /tmp/pip-req-build-teagkvn6
  Running command git clone -q https://github.com/kassemm/rl-agents /tmp/pip-req-build-teagkvn6
  Created wheel for rl-agents: filename=rl_agents-1.0.dev0-cp37-none-any.whl size=114548 sha256=fa3aac832f338e235f71c804beba81e65bcee1b88ade3dfb1e2dc8d69dbad4c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-_aayi4q7/wheels/17/93/27/d9e1db6f2c186cff3f330902d3380e247a77b432afeb6bc94d
Successfully built rl-agents
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Reading package lists... Done
Building dependency tree       
Reading st

In [2]:
%cd /content/rl-agents/scripts/
env_config = 'configs/HighwayEnv/env.json'
agent_config = 'configs/HighwayEnv/agents/DQNAgent/dqn.json' ##Agent is configured to act greedy
f=open(agent_config,"r");
print(f.read())

/content/rl-agents/scripts
{
    "__class__": "<class 'rl_agents.agents.deep_q_network.pytorch.DQNAgent'>",
    "model": {
        "type": "MultiLayerPerceptron",
        "layers": [256, 256]
    },
    "double": false,
    "gamma": 0.8,
    "n_steps": 1,
    "batch_size": 32,
    "memory_capacity": 15000,
    "target_update": 50,
    "exploration": {
        "method": "Greedy",
        "tau": 6000,
        "temperature": 1.0,
        "final_temperature": 0.05
    },
    "loss_function": "l2"
}



## Training

Prepare environment, agent, and evaluation process.


In [8]:


env = load_environment(env_config)
import pprint
from matplotlib import pyplot as plt
env.config["lane_change_reward"] = -0.75  ##-ve value
env.config["vehicles_count"] = 10
env.config["right_lane_reward"]=0.1  #increase little bit the reward for chaning to the right.
#env.config['reward_speed_range'] = [0, 100]
env.config["lanes_count"] = 2
env.config["initial_lane_id"] = 0
pprint.pprint(env.config)
print("Agent Config")
pprint.pprint(agent_config)


env.reset()
plt.imshow(env.render(mode="rgb_array"))
plt.show()

agent = load_agent(agent_config, env)
evaluation = Evaluation(env, agent, num_episodes=10, display_env=False)
print(f"Ready to train {agent} on {env}")

INFO: Making new env: highway-v0
{'action': {'type': 'DiscreteMetaAction'},
 'centering_position': [0.3, 0.5],
 'collision_reward': -1,
 'controlled_vehicles': 1,
 'duration': 40,
 'ego_spacing': 2,
 'high_speed_reward': 0.4,
 'id': 'highway-v0',
 'import_module': 'highway_env',
 'initial_lane_id': 0,
 'lane_change_reward': -0.75,
 'lanes_count': 2,
 'manual_control': False,
 'observation': {'type': 'Kinematics'},
 'offroad_terminal': False,
 'offscreen_rendering': False,
 'other_vehicles_type': 'highway_env.vehicle.behavior.IDMVehicle',
 'policy_frequency': 1,
 'real_time_rendering': False,
 'render_agent': True,
 'reward_speed_range': [20, 30],
 'right_lane_reward': 0.1,
 'scaling': 5.5,
 'screen_height': 150,
 'screen_width': 600,
 'show_trajectories': False,
 'simulation_frequency': 15,
 'vehicles_count': 10,
 'vehicles_density': 1}
Agent Config
'configs/HighwayEnv/agents/DQNAgent/dqn.json'


error: ignored

Start training. Run tensorboard locally to visualize training.

In [4]:
#tensorboard --logdir "{evaluation.directory}"
evaluation.train()

[INFO] Episode 0 score: 6.1 
[INFO] Episode 1 score: 2.7 
[INFO] Episode 2 score: 7.1 
[INFO] Episode 3 score: 10.5 
[INFO] Episode 4 score: 2.7 
[INFO] Episode 5 score: 2.8 
[INFO] Episode 6 score: 7.0 
[INFO] Episode 7 score: 5.3 
[INFO] Episode 8 score: 9.7 
[INFO] Episode 9 score: 11.3 
[INFO] Saved DQNAgent model to /content/rl-agents/scripts/out/HighwayEnv/DQNAgent/run_20210524-083809_58/checkpoint-final.tar 


INFO: Finished writing results. You can upload them to the scoreboard via gym.upload('/content/rl-agents/scripts/out/HighwayEnv/DQNAgent/run_20210524-083809_58')


In [5]:
 #gym.upload('/content/rl-agents/scripts/out/HighwayEnv/DQNAgent/run_20210523-212333_58')

Progress can be visualised in the tensorboard cell above, which should update every 30s (or manually). You may need to click the *Fit domain to data* buttons below each graph.

## Testing

Run the learned policy for a few episodes.

In [6]:
#env = load_environment(env_config)
"""
agent_config = {
    "__class__": "<class 'rl_agents.agents.tree_search.deterministic.DeterministicPlannerAgent'>",
    "env_preprocessors": [{"method":"simplify"}],
    "budget": 50,
    "gamma": 0.7,
}

"""


env.configure({"offscreen_rendering": True})
agent = load_agent(agent_config, env)
evaluation = Evaluation(env, agent, num_episodes=7, recover=True)
evaluation.test()
show_videos(evaluation.run_directory)

[INFO] Choosing GPU device: 0, memory used: 899 
[INFO] Loaded DQNAgent model from out/HighwayEnv/DQNAgent/saved_models/latest.tar 


INFO: Creating monitor directory out/HighwayEnv/DQNAgent/run_20210524-083816_58
INFO: Starting new video recorder writing to /content/rl-agents/scripts/out/HighwayEnv/DQNAgent/run_20210524-083816_58/openaigym.video.1.58.video000000.mp4


[INFO] Episode 0 score: 8.0 


INFO: Starting new video recorder writing to /content/rl-agents/scripts/out/HighwayEnv/DQNAgent/run_20210524-083816_58/openaigym.video.1.58.video000001.mp4


[INFO] Episode 1 score: 6.1 


...

INFO: Starting new video recorder writing to /content/rl-agents/scripts/out/HighwayEnv/DQNAgent/run_20210524-083816_58/openaigym.video.1.58.video000005.mp4


[INFO] Episode 5 score: 4.4 


INFO: Starting new video recorder writing to /content/rl-agents/scripts/out/HighwayEnv/DQNAgent/run_20210524-083816_58/openaigym.video.1.58.video000006.mp4


[INFO] Episode 6 score: 12.2 


INFO: Finished writing results. You can upload them to the scoreboard via gym.upload('/content/rl-agents/scripts/out/HighwayEnv/DQNAgent/run_20210524-083816_58')
